# LeNet
 - [Tensorflow Dataset](https://www.tensorflow.org/datasets/catalog/overview)



---


---


### MNIST dataset  (Gray images)
 - Dataset Perparation

In [1]:
import tensorflow as tf
print(f"Tensorflow Version : {tf.__version__}\n")

from tensorflow.keras.datasets import mnist

# dataset = mnist.load_data()
# print(f"train dataset : {len(dataset[0][0])}")
# print(f"train labels : {len(set(dataset[0][1]))}")
# print(f"{set(dataset[0][1])}\n")

# print(f"test dataset : {len(dataset[1][0])}")
# print(f"test labels : {len(set(dataset[1][1]))}")
# print(f"{set(dataset[1][1])}\n")

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

train_images = train_images[..., tf.newaxis].astype("float32")
test_images = test_images[..., tf.newaxis].astype("float32")

train_labels = train_labels.astype('int32')
test_labels = test_labels.astype('int32')

print(f"train dataset (images) : {train_images.shape}")
print(f"train dataset (labels) : {train_labels.shape}\n")

print(f"test dataset (images) : {test_images.shape}")
print(f"test dataset (labels) : {test_labels.shape}\n")

print(f"the number of label categories : {len(set(train_labels))}")
print(f"{set(train_labels)}\n")

Tensorflow Version : 2.13.0

train dataset (images) : (60000, 28, 28, 1)
train dataset (labels) : (60000,)

test dataset (images) : (10000, 28, 28, 1)
test dataset (labels) : (10000,)

the number of label categories : 10
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}



In [2]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(100).batch(32)
print(f"train batch 개수 : {len(train_ds)}\n")

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).shuffle(10).batch(32)
print(f"test batch 개수 : {len(test_ds)}\n")

for i, (images, labels) in enumerate(train_ds):
  if i == 0:
    print(f"images shape (batch{i}) : {images.shape}\n")
    print(f"labels shape (batch{i}) : {labels.shape}\n")
    break

train batch 개수 : 1875

test batch 개수 : 313

images shape (batch0) : (32, 28, 28, 1)

labels shape (batch0) : (32,)



In [3]:
# import numpy as np

# (train_images, train_labels), _ = mnist. load_data()
# train_images = np.expand_dims(train_images, axis=3).astype(np.float32)
# train_labels = train_labels.astype(np. int32)

# print(f"train dataset (images) : {train_images.shape}")
# print(f"train dataset (labels) : {train_labels.shape}\n")

# train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(100).batch(32)
# print(train_ds, end='\n\n')

# print(f"batch 개수 : {len(train_ds)}\n")

# for i, (images, labels) in enumerate(train_ds):
#   if i == 1874 or i == 0:
#     print(f"images shape (batch{i}) : {images.shape}\n")
#     print(f"labels shape (batch{i}) : {labels.shape}\n")
#   else:
#     continue



---


### Implementation with mnist

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


class LeNet_mnist(Model):

  def __init__(self):
    super(LeNet_mnist, self).__init__()

    self.fe = Sequential()
    self.fe.add(Conv2D(filters=1, kernel_size=1))
    self.fe.add(Conv2D(filters=6, kernel_size=5, padding='same', activation='tanh'))
    self.fe.add(AveragePooling2D(pool_size=2, strides=2))
    self.fe.add(Conv2D(filters=16, kernel_size=5, padding='valid', activation='tanh'))
    self.fe.add(AveragePooling2D(pool_size=2, strides=2))
    self.fe.add(Conv2D(filters=120, kernel_size=5, padding='valid', activation='tanh'))
    self.fe.add(Flatten())

    self.classifier = Sequential()
    self.classifier.add(Dense(units=84, activation='tanh'))
    self.classifier.add(Dense(units=10, activation='softmax'))

  def call(self, x):

    print(f"Input shape : {x.shape}")
    x = self.fe(x)
    x = self.classifier(x)
    print(f"Output shape : {x.shape}")

    return x


model = LeNet_mnist()

In [4]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

loss_object = SparseCategoricalCrossentropy(from_logits=False)
optimizer = Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy(name='test_accuracy')

In [5]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:

    predictions = model(images)
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [6]:
@tf.function
def test_step(images, labels):

  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


In [8]:
EPOCHS = 100

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
      f'Epoch {epoch+1}, '
      f'Loss : {train_loss.result()}, '
      f'Accuracy : {100-train_accuracy.result()*100}, '
      f'Test Loss : {test_loss.result()}, '
      f'Test Accuracy : {100-test_accuracy.result()*100}')

Epoch 1, Loss : 0.03621170297265053, Accuracy : 96.37882995605469, Test Loss : 0.06347224861383438, Test Accuracy : 93.64371490478516
Epoch 2, Loss : 0.030185967683792114, Accuracy : 96.98139953613281, Test Loss : 0.06635253131389618, Test Accuracy : 93.35417938232422
Epoch 3, Loss : 0.025027988478541374, Accuracy : 97.49720001220703, Test Loss : 0.06585550308227539, Test Accuracy : 93.4076156616211
Epoch 4, Loss : 0.021440360695123672, Accuracy : 97.85596466064453, Test Loss : 0.06466184556484222, Test Accuracy : 93.52350616455078
Epoch 5, Loss : 0.018732598051428795, Accuracy : 98.12673950195312, Test Loss : 0.06357070058584213, Test Accuracy : 93.63278198242188
Epoch 6, Loss : 0.01731489785015583, Accuracy : 98.26850891113281, Test Loss : 0.060509391129016876, Test Accuracy : 93.93939208984375
Epoch 7, Loss : 0.014801131561398506, Accuracy : 98.51988983154297, Test Loss : 0.05665688216686249, Test Accuracy : 94.32526397705078
Epoch 8, Loss : 0.013269477523863316, Accuracy : 98.67304

In [ ]:
# from tensorflow.keras.losses import SparseCategoricalCrossentropy

# model_mnist = LeNet_mnist()
# losses_mnist = {}

# #####Forward Propagation######
# for i, (images, labels) in enumerate(train_ds):
#   predictions = model_mnist(images)

#   loss_scce_func = SparseCategoricalCrossentropy()
#   loss_scce = loss_scce_func(labels, predictions)

#   losses_mnist[i] = loss_scce

#   break

# losses_mnist

Input shape : (32, 28, 28, 1)
Output shape : (32, 10)


{0: <tf.Tensor: shape=(), dtype=float32, numpy=2.3185973>}



---


---

### Cifar100 dataset (RGB images)
 - Dataset Perparation

In [ ]:
from tensorflow.keras.datasets import cifar100
import numpy as np

(train_images, train_labels), _ = cifar100.load_data()
train_images = train_images.astype(np.float32)
train_labels = train_labels.astype(np.int32)

print(f"train dataset (images) : {train_images.shape}")
print(f"train dataset (labels) : {train_labels.shape}\n")
print(f"labels : {len(np.unique(train_labels))}\n")


train dataset (images) : (50000, 32, 32, 3)
train dataset (labels) : (50000, 1)

labels : 100




---

### Implementation with cifar100

In [ ]:
batch_size = 32

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.batch(batch_size)

print(train_ds, end='\n\n')

print(f"batch 개수 : {len(train_ds)}\n")

# for i, (images, labels) in enumerate(train_ds):
#   if i == 1562 or i == 0:
#     print(f"images shape (batch{i}) : {images.shape}\n")
#     print(f"labels shape (batch{i}) : {labels.shape}\n")
#   else:
#     continue

<_BatchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))>

batch 개수 : 1563



In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


class LeNet_cifar100(Model):

  def __init__(self):
    super(LeNet_cifar100, self).__init__()

    self.fe = Sequential()
    self.fe.add(Conv2D(filters=1, kernel_size=1))
    self.fe.add(Conv2D(filters=8, kernel_size=5, padding='same', activation='tanh'))
    self.fe.add(AveragePooling2D(pool_size=2, strides=2))
    self.fe.add(Conv2D(filters=64, kernel_size=5, padding='valid', activation='tanh'))
    self.fe.add(AveragePooling2D(pool_size=2, strides=2))
    self.fe.add(Conv2D(filters=512, kernel_size=5, padding='valid', activation='tanh'))
    self.fe.add(Flatten())

    self.classifier = Sequential()
    self.classifier.add(Dense(units=256, activation='tanh'))
    self.classifier.add(Dense(units=100, activation='softmax'))

  def call(self, x):

    print(f"Input shape : {x.shape}")
    x = self.fe(x)
    x = self.classifier(x)
    print(f"Output shape : {x.shape}")

    return x


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model_cifar100 = LeNet_cifar100()
losses_cifar100 = {}

#####Forward Propagation######
for i, (images, labels) in enumerate(train_ds):
  predictions = model_cifar100(images)

  loss_scce_func = SparseCategoricalCrossentropy()
  loss_scce = loss_scce_func(labels, predictions)

  losses_cifar100[i] = loss_scce

  break

losses_cifar100

Input shape : (32, 32, 32, 3)
Output shape : (32, 100)


{0: <tf.Tensor: shape=(), dtype=float32, numpy=4.6056185>}